In [1]:
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack, hstack, join
from astropy.io import fits
import fitsio
from scipy import stats
import sys, os, warnings
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict, train_test_split, KFold

### Process:

- group the cleaned table by obsday
- for each obsday add a column that contains an array of mirror_desired_temp and an array of mirror_cooling throughout that obsday
- go backwards in mirror cooling until it is on and hits the first "off". Save that index and return the mirror_desired_temp at that index

In [2]:
combined_df = Table(fitsio.read('outputs/full_cleaned_telemetry.fits')).to_pandas()

In [3]:
combined_df['Binned Time'] = pd.to_datetime(combined_df['Binned Time'])
combined_df['Observing Day'] = combined_df['Binned Time'] - pd.Timedelta(hours=12)
combined_df['Observing Day'] = combined_df['Observing Day'].dt.floor('D') + pd.Timedelta(hours=12)

In [4]:
combined_df

,mirror_ready,airmass,temperature,pressure,humidity,wind_speed,wind_direction,dimm,dewpoint,mirror_avg_temp,mirror_desired_temp,mirror_temp,mirror_cooling,air_temp,air_flow,air_dewpoint,Binned Time,Observing Day
0,1.0,1.0,22.259740,794.30000,31.119481,11.488312,281.236364,0.762155,4.346753,20.879634,20.0,20.891604,0.0,21.885050,0.000000,5.245020,2021-09-15 02:20:00,2021-09-14 12:00:00
1,1.0,1.0,22.160396,794.30495,31.698020,12.920792,261.993069,0.780386,4.519802,20.975901,20.0,20.995010,0.0,21.885861,0.000000,5.226505,2021-09-15 02:30:00,2021-09-14 12:00:00
2,1.0,1.0,22.024752,794.40000,32.000000,10.909901,262.174257,0.862771,4.524752,21.065554,20.0,21.079604,0.0,21.820079,0.000000,5.242040,2021-09-15 02:40:00,2021-09-14 12:00:00
3,1.0,1.0,22.000000,794.40000,32.000000,10.552475,273.168317,0.920535,4.500000,21.112683,20.0,21.106188,0.0,21.697297,0.000000,5.246446,2021-09-15 02:50:00,2021-09-14 12:00:00
4,1.0,1.0,21.925743,794.59505,32.247525,12.011881,288.204950,0.899641,4.549505,21.112891,20.0,21.125941,0.0,21.601574,0.000000,5.291406,2021-09-15 03:00:00,2021-09-14 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62681,0.0,1.0,-3.790291,795.00000,100.000000,142.600000,269.000000,0.000000,-3.790291,6.207466,4.0,6.197660,1.0,3.872291,1.136417,-2.344262,2023-02-22 20:40:00,2023-02-22 12:00:00
62682,0.0,1.0,-3.900000,795.00000,100.000000,142.600000,269.000000,0.000000,-3.900000,6.109534,4.0,6.096117,1.0,3.816359,1.135000,-2.366136,2023-02-22 20:50:00,2023-02-22 12:00:00
62683,0.0,1.0,-4.009709,795.00000,100.000000,142.600000,269.000000,0.000000,-4.009709,6.093990,4.0,6.110204,1.0,3.829573,1.137583,-2.409981,2023-02-22 21:00:00,2023-02-22 12:00:00
62684,0.0,1.0,-4.050485,795.00000,100.000000,142.600000,269.000000,0.000000,-4.050485,6.106981,4.0,6.105673,1.0,3.806981,1.138096,-2.449231,2023-02-22 21:10:00,2023-02-22 12:00:00


In [9]:
# Perform Obsday grouping
# Convert 'Observing Day' to datetime format
df = combined_df
#df['Observing Day'] = pd.to_datetime(df['Observing Day'])

# Group by 'Observing Day' and apply the custom aggregation function
cooling_arrays = df.groupby('Observing Day')['mirror_cooling'].apply(list).reset_index()
desired_temp_arrays = df.groupby('Observing Day')['mirror_desired_temp'].apply(list).reset_index()
desired_temp_arrays

,Observing Day,mirror_desired_temp
0,2021-09-14 12:00:00,"[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20...."
1,2021-09-15 12:00:00,"[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20...."
2,2021-09-16 12:00:00,"[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20...."
3,2021-09-17 12:00:00,"[18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18...."
4,2021-09-18 12:00:00,"[18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18.0, 18...."
...,...,...
449,2023-02-18 12:00:00,"[-3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3.0, -3...."
450,2023-02-19 12:00:00,"[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, ..."
451,2023-02-20 12:00:00,"[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, ..."
452,2023-02-21 12:00:00,"[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, ..."


In [6]:
# Returns the first index of last nonzero group given an array of groups of 0 and nonzero values
def first_index_of_last_nonzero_group(arr):
    last_nonzero_group_start = None

    for i in range(len(arr) - 1, -1, -1):
        if arr[i] == 1:
            last_nonzero_group_start = i
            break

    if last_nonzero_group_start is not None:
        while last_nonzero_group_start >= 0 and arr[last_nonzero_group_start] == 1:
            last_nonzero_group_start -= 1

        return last_nonzero_group_start + 1 + 2 #+2 for 20 mins buffer
    else:
        return 0

In [7]:
indices = cooling_arrays['mirror_cooling'].apply(first_index_of_last_nonzero_group)

In [68]:
data = {'Cooling Indices': indices,
        'Desired Temperature Arrays': desired_temp_arrays['mirror_desired_temp']}

df = pd.DataFrame(data)

# Function to access the corresponding index from the array
def get_value_from_array(row):
    index = int(row['Cooling Indices'])
    array = row['Desired Temperature Arrays']
    if 0 <= index < len(array):
        return array[index]
    else:
        return None

# Apply the function to create a new column 'Corresponding_Value'
df['Obs Day Desired Temp'] = df.apply(lambda row: get_value_from_array(row), axis=1)


In [71]:
real_desired_temps = df['Obs Day Desired Temp']

In [92]:
def consecutive_group_length_less_than_5(arr):
    count = 0
    last_value = arr[0]

    for value in arr:
        if value == last_value:
            count += 1
        else:
            if count < 30:
                return False  # Return False if any consecutive group has length less than 5
            count = 1  # Start a new group
            last_value = value

        

    return True  # Return True if no consecutive group has length 5 or more

# Example usage:
arr1 = [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1]
arr2 = [1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1]
arr3 = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
    
result1 = consecutive_group_length_less_than_5(arr1)
result2 = consecutive_group_length_less_than_5(arr2)

print(result1)  # Output: False
print(result2)  # Output: True
print(consecutive_group_length_less_than_5(arr3))

False
False
False


In [93]:
# contains True if it's a "good" observing day (where the mirror cooling didn't toggle too much)
is_good = cooling_arrays['mirror_cooling'].apply(consecutive_group_length_less_than_5)

In [94]:
data = {'is_good_obsday': is_good,
        'adjusted_desired_temp': real_desired_temps}

temps_df = pd.DataFrame(data)
fits_table = Table.from_pandas(temps_df)
fits_table.write('outputs/desiredtemps.fits', format='fits', overwrite=True)